In [1]:

import os
# import random
# import shutil
# import time
import warnings
import sys
sys.path.append('../')

import torch
import torch.nn as nn
# import torch.nn.parallel
# import torch.backends.cudnn as cudnn
# import torch.distributed as dist

import torch.optim
# import torch.multiprocessing as mp
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# import torchvision.models as models
from resnet import *

from main import *

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

In [2]:
# args = parser.parse_args(args=[])
# args = parser.parse_args()
import easydict 
args = easydict.EasyDict({ "batch-size": 256, 
                          "epochs": 100,
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':3,
                         'saved_dir':'../trained_model/model_best.pt'})


In [3]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)
print ('Current cuda device ', torch.cuda.current_device()) # check
# 4번 디바이스만 이용하려면 "4"를 입력
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print ('Current cuda device ', torch.cuda.current_device()) # check

8
cuda:3
Current cuda device  3


In [4]:
# import numpy as np
# imagenet_embeding = np.load('../data/imagenet_embeding.npy')
# imagenet_embeding.shape

In [5]:
# imagenet_embeding = torch.tensor(imagenet_embeding)
# imagenet_embeding.shape

In [6]:
print("=> using pre-trained model '{}'".format('resnet18'))
# model = models.__dict__['resnet18'](pretrained=True)
# model = models.resnet18(pretrained=False)
model = resnet18(pretrained=False)
# model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)
# optimizer = torch.optim.Adam(
#     model.parameters(), lr=0.1)


=> using pre-trained model 'resnet18'


In [7]:
model.fc.weight[0][0]

tensor(-0.0302, grad_fn=<SelectBackward>)

In [8]:
# torch.save(model.state_dict(), '../trained_model/init.pt')
model.load_state_dict(torch.load('../trained_model/init.pt'))

<All keys matched successfully>

In [9]:
model.fc.weight[0][0]

tensor(-0.0301, grad_fn=<SelectBackward>)

In [10]:
# model_dict = model.state_dict() 
# for k in model_dict :
#     if 'fc.weight' in k :
#         print(model_dict[k].shape)
#         print(model_dict[k])
#         model_dict[k] = imagenet_embeding
# #     if 'fc.bias' in k :
# #         print(model_dict[k])        
# #     print(model_dict[k].shape)
# model.load_state_dict(model_dict)
# model_dict = model.state_dict() 
# for k in model_dict :
#     if 'fc.weight' in k :
# #         print(model_dict[k].shape)
#         print(model_dict[k])

In [11]:
# model.fc.weight.requires_grad = False

In [12]:
# for p in model.parameters() :
#     print(p)

In [13]:
# model.eval()

In [14]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [16]:
# next(iter(train_dataset))

In [17]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None

In [18]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=(train_sampler is None),
    num_workers=8, pin_memory=True, sampler=train_sampler)

In [19]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=4, pin_memory=True)

In [20]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    adjust_learning_rate(optimizer, epoch, args.lr)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args, device)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args, device)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best, args, filename='../trained_model/checkpoint.pt')    

/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][   0/5005]	Time 60.624 (60.624)	Data 34.235 (34.235)	Loss 7.0416e+00 (7.0416e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][1001/5005]	Time  0.379 ( 1.922)	Data  0.000 ( 1.361)	Loss 5.6062e+00 (6.1895e+00)	Acc@1   3.52 (  1.99)	Acc@5  14.84 (  6.97)
Epoch: [0][2002/5005]	Time  0.365 ( 2.160)	Data  0.000 ( 1.491)	Loss 4.9209e+00 (5.7257e+00)	Acc@1   7.81 (  4.45)	Acc@5  22.27 ( 13.16)
Epoch: [0][3003/5005]	Time  0.385 ( 2.411)	Data  0.000 ( 1.617)	Loss 4.5214e+00 (5.3751e+00)	Acc@1  14.84 (  7.14)	Acc@5  34.38 ( 18.74)
Epoch: [0][4004/5005]	Time  0.379 ( 2.330)	Data  0.000 ( 1.593)	Loss 4.0172e+00 (5.0981e+00)	Acc@1  20.70 (  9.68)	Acc@5  42.97 ( 23.49)
 * Acc@1 22.670 Acc@5 46.460
************train_loss 4.8716156125902295 val_acc 22.670000076293945*************
Epoch: [1][   0/5005]	Time 34.107 (34.107)	Data 23.164 (23.164)	Loss 3.9349e+00 (3.9349e+00)	Acc@1  21.88 ( 21.88)	Acc@5  44.14 ( 44.14)
Epoch: [1][1001/5005]	Time  0.387 ( 2.504)	Data  0.000 ( 1.641)	Los